In [38]:
##Importing Libraries and created local vector db

from typing import List, Optional, Tuple, Dict
import gradio as gr
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from pypdf.errors import FileNotDecryptedError
# Globals
vectorstore: Optional[FAISS] = None
qa_chain: Optional[ConversationalRetrievalChain] = None


In [39]:
# Retrieve OpenAI API key from .env file
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [41]:
def load_document(file_path: str) -> Tuple[Optional[List], Optional[str]]:
    """
    Load and process a document from the given file path.

    Parameters:
        file_path (str): The path to the document file.

    Returns:
        tuple: A list of documents and an error message (if any).
    """
    try:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith('.txt'):
            loader = TextLoader(file_path)
        else:
            return None, "Unsupported file format"

        documents = loader.load()
        return documents, None

    except FileNotDecryptedError:
        return None, "Error: The PDF file is password-protected and cannot be processed."
    except Exception as e:
        return None, f"Error: {str(e)}"

In [42]:
def create_vectorstore(documents: List) -> FAISS:
    """
    Create a vectorstore from the provided documents using OpenAI embeddings.
    """
    embeddings = OpenAIEmbeddings()
    return FAISS.from_documents(documents, embeddings)


In [43]:
def upload_and_process(file: gr.File) -> str:
    """
    Handle file upload, process the document, and initialize the QA chain.

    Parameters:
        file (gr.File): The uploaded file to process.

    Returns:
        str: Status message indicating success or failure.
    """
    global vectorstore, qa_chain

    if not file:
        return "Error: No document uploaded. Please upload a file."

    documents, error = load_document(file.name)
    if error:
        return error  # Display the error returned by load_document

    vectorstore = create_vectorstore(documents)
    llm = OpenAI(temperature=0.5)
    qa_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

    return "Document processed and ready for chat!"


In [44]:
def chat_with_document(query: str, history: List[Dict[str, str]]) -> List[Dict[str, str]]:
    """
    Handle user queries by interacting with the uploaded document.
    """
    if not vectorstore or not qa_chain:
        history.append({"role": "assistant", "content": "Please upload and process a document first."})
        return history

    # Prepare history for the ConversationalRetrievalChain
    formatted_history = [(entry["role"], entry["content"]) for entry in history]

    inputs = {"question": query, "chat_history": formatted_history}
    result = qa_chain.invoke(inputs)

    # Update chat history
    history.append({"role": "user", "content": query})
    history.append({"role": "assistant", "content": result['answer']})

    return history


In [45]:
with gr.Blocks() as demo:
    # Header Section
    gr.Markdown(
        """
        <h1 style="text-align: center;">📚 ChatBridge</h1>
        <h3 style="text-align: center; color: gray;">Effortlessly Engage with Your Documents</h3>
        """, 
        elem_id="header"
    )

    # Instructions Section
    gr.Markdown(
        """
        <p><b>How to Use:</b></p>
        <ol>
            <li>Upload your document by dragging and dropping it below.</li>
            <li>Click "Analyze Document" to process the uploaded file.</li>
            <li>Type your questions in the chat box and click "Chat" to interact with the document.</li>
        </ol>
        <p style="color: gray; font-size: small;">Supported Formats: PDF, Word, Text</p>
        """,
        elem_id="instructions"
    )

    # File Upload and Status
    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            file_input = gr.File(label="📂 Drag & Drop Your File", file_types=[".pdf", ".docx", ".txt"])
            upload_button = gr.Button("📊 Analyze Document", variant="primary")
        with gr.Column(scale=2, min_width=400):
            status_output = gr.Textbox(
                label="Status",
                interactive=False,
                placeholder="Upload a document to start!",
                lines=2,
            )

    # Chat Section
    with gr.Row():
        query_input = gr.Textbox(
            label="💬 Ask Anything",
            placeholder="Type your question about the document here...",
            lines=2,
        )
        submit_button = gr.Button("💬 Chat", variant="primary")

    # Chat History Section
    chat_history = gr.Chatbot(label="🗨️ Conversation History", type="messages", height=400)

    # Footer
    gr.Markdown(
        """
        <p style="text-align: center; color: gray; font-size: small;">Created by <b>Prithvi</b></p>
        """,
        elem_id="footer"
    )

    # Handle file upload
    upload_button.click(upload_and_process, inputs=file_input, outputs=status_output)

    # Handle chat functionality
    def chat_and_clear(query, chat_history):
        updated_chat = chat_with_document(query, chat_history)
        return updated_chat, ""  # Clear input after each query

    submit_button.click(
        chat_and_clear,
        inputs=[query_input, chat_history],
        outputs=[chat_history, query_input],
    )


In [46]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://16e820c63c57eef95d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
demo.close()

Closing server running on port: 7861
